## Descriptors

- [Language Reference](https://docs.python.org/2/reference/datamodel.html#implementing-descriptors)
- [Descriptors How To](https://docs.python.org/2/howto/descriptor.html)
    - [In Chinese](http://pyzh.readthedocs.org/en/latest/Descriptor-HOW-TO-Guide.html)
- [A post in Chinese](http://www.jianshu.com/p/250f0d305c35)

__Note:__ 

- Descriptors are only invoked for new style objects or classes
- A class is new style if it inherits from __object__ or __type__

### Definition

- In general, a descriptor is an object attribute with “binding behavior”, one whose attribute access has been overridden by methods in the descriptor protocol. 
- Those methods are ```__get__()```, ```__set__()```, and ```__delete__()```. 
- If any of those methods are defined for an object, it is said to be a descriptor.

所谓描述器，即实现了描述符协议，即```__get__, __set__, 和 __delete__```方法的对象。

#### Attribute Access

- The default behavior for attribute access is to ___get, set, or delete___ the attribute from an object’s dictionary. 
- For instance, __a.x__ has a lookup chain:
    - ```a.__dict__['x']```, then 
    - ```type(a).__dict__['x']```, and continuing through 
    - the base classes of type(a) excluding metaclasses. 
- If the looked-up value is an object with one of the descriptor methods, then Python may:
    - override the default behavior and 
    - invoke the descriptor method instead. 
    

#### Descriptor Protocol

- Descriptors are a powerful, general purpose protocol. 
- They are the mechanism behind:
    - properties, 
    - methods, 
    - static methods, 
    - class methods, and 
    - super(). 
- They are used throughout Python itself to implement the new style classes introduced in version 2.2. 
- Descriptors simplify the underlying C-code and offer a flexible set of new tools for everyday Python programs.

```
descr.__get__(self, obj, type=None) --> value
descr.__set__(self, obj, value)     --> None
descr.__delete__(self, obj)         --> None
```

##### Data Descriptor
```
If an object defines both __get__() and __set__(), it is considered a data descriptor.
```
##### Non-Data Descriptor
```
- Descriptors that only define __get__() are called non-data descriptors.
- They are typically used for methods but other uses are possible.
```

To make a __read-only data descriptor__, define both ```__get__() and __set__() with the __set__()``` raising an __AttributeError__ when called. 

Defining the ```__set__()``` method with an exception raising placeholder is enough to make it a data descriptor.


### Invoking Descriptors

- A descriptor can be called directly by its method name. For example, **d.\_\_get\_\_**(obj).

- More common for a descriptor to be invoked upon attribute access. 
    - **obj.d** looks up d in the dictionary of **obj**
    - If **d** defines the method **\_\_get\_\_()**, then **d.\_\_get\_\_()** is invoked: 
        - if **obj** is an object:
            - **object.\_\_getattribute\_\_()** 
            - **b.x** ==> **type(b).\_\_dict\_\_['x'].\_\_get\_\_(b, type(b))**
            - precedence chain:
                data descriptors > instance variables > non-data descriptors > \_\_getattr\_\_() 
        - if **obj** is an class:
            - **type.\_\_getattribute\_\_()** 
            - **B.x** ==> **B.\_\_dict\_\_['x'].\_\_get\_\_(None, B)**
            - In pure Python, it looks like:
            > 
              ```
              def __getattribute__(self, key):
                """Emulate type_getattro() in Objects/typeobject.c"""
                v = object.__getattribute__(self, key)
                if hasattr(v, '__get__'):
                    return v.__get__(None, self)
                return v
              ```



    
- The important points are:


    - descriptors are invoked by the `__getattribute__()` method
    - overriding `__getattribute__()` prevents automatic descriptor calls
    - `__getattribute__()` is only available with new style classes and objects
    - `object.__getattribute__()` and `type.__getattribute__()` make different calls to `__get__()`.
    - data descriptors always override instance dictionaries.
    - non-data descriptors may be overridden by instance dictionaries.
    - The object returned by super() also has a custom `__getattribute__()` method for invoking descriptors. The call super(B, obj).m() searches obj.__class__.__mro__ for the base class A immediately following B and then returns A.__dict__['m'].__get__(obj, B). If not a descriptor, m is returned unchanged. If not in the dictionary, m reverts to a search using object.__getattribute__().

Note, in Python 2.2, super(B, obj).m() would only invoke __get__() if m was a data descriptor. In Python 2.3, non-data descriptors also get invoked unless an old-style class is involved. The implementation details are in super_getattro() in Objects/typeobject.c.

The details above show that the mechanism for descriptors is embedded in the __getattribute__() methods for object, type, and super(). Classes inherit this machinery when they derive from object or if they have a meta-class providing similar functionality. Likewise, classes can turn-off descriptor invocation by overriding __getattribute__().

In [84]:
"""
Example 1: Data Descriptor
"""
class WebFramework(object):
    
    def __init__(self, name='Flask'):
        self.name = name

    def __get__(self, instance, owner):
        return self.name

    def __set__(self, instance, value):
        self.name = value


class PythonSite(object):

    webframework = WebFramework()

print PythonSite.webframework       # Flask
PythonSite.webframework = 'Tornado'
print PythonSite.webframework       # Tornado


class PythonSite(object):

    webframework = WebFramework()

    version = 0.01

    def __init__(self, site):
        self.site = site


pysite = PythonSite('ghost')
print
print vars(PythonSite).items() # webframework，version两个属性，以及 __init__方法
print
print vars(pysite)             # 后者仅有一个site属性
print
print PythonSite.__dict__


Flask
Tornado

[('__module__', '__main__'), ('version', 0.01), ('__dict__', <attribute '__dict__' of 'PythonSite' objects>), ('webframework', <__main__.WebFramework object at 0x0000000003993358>), ('__weakref__', <attribute '__weakref__' of 'PythonSite' objects>), ('__doc__', None), ('__init__', <function __init__ at 0x0000000003932438>)]

{'site': 'ghost'}

{'__module__': '__main__', 'version': 0.01, '__dict__': <attribute '__dict__' of 'PythonSite' objects>, 'webframework': <__main__.WebFramework object at 0x0000000003993358>, '__weakref__': <attribute '__weakref__' of 'PythonSite' objects>, '__doc__': None, '__init__': <function __init__ at 0x0000000003932438>}


In [95]:
class PythonSite(object):

    webframework = WebFramework() # class property

    version = 0.01                # class property

    def __init__(self, site):
        self.site = site          # instance property
        
pysite1 = PythonSite('ghost')
pysite2 = PythonSite('admin')
pysite3 = PythonSite('guest')

try:
    print 'version' in vars(pysite3)
except Exception as e:
    print e
    
print PythonSite.version # 0.01
print pysite1.version    # 0.01
print pysite2.version    # 0.01
print pysite1.version is pysite2.version # True
pysite1.version = 'pysite1' # 当pysite1修改了version，实际上是给自己添加了一个version属性。类属性并没有被改变。
print vars(pysite1)         # {'version': 'pysite1', 'site': 'ghost'}
print vars(pysite2)         # site': 'ghost'}
PythonSite.version = 0.02   # 当PythonSite改变了version属性的时候，pysite2的该属性也对应被改变。
print pysite1.version       # pysite1
print pysite2.version       # 0.02

try:
    print 'version' in pysite3.__dict__
except Exception as e:
    print e
    
print type(pysite3).__dict__['version']
type(pysite3).__dict__['webframework']
type(pysite1).__dict__['webframework'].__get__(None, PythonSite)


False
0.01
0.01
0.01
True
{'version': 'pysite1', 'site': 'ghost'}
{'site': 'admin'}
pysite1
0.02
False
0.02


'Flask'

In [77]:
"""
Example 2: Read Only Data Descriptor
"""
class WebFramework2(object):
    def __init__(self, name='Tornado'):
        self.name = name

    def __get__(self, instance, owner):
        return self.name

    def __set__(self, instance, value):
        raise AttributeError('This is a read-only data descriptor.')


class PythonSite2(object):

    webframework = WebFramework2()

a = PythonSite2()
print a.webframework       # Flask
print a.__getattribute__('webframework')
    # === a.webframework
print type(a).__dict__['webframework'].__get__(a, type(a)) 
    # === a.webframework
    # === a.__getattribute__('webframework')


try:
    a.webframework = 'Flask'
    print a.webframework
except AttributeError as e:
    print e
    
try:
    PythonSite2.webframework = 'Flask'
    print PythonSite2.webframework
except AttributeError as e:
    print e
    
try:
    a.webframework = 'Django'
    print id(type(a).__dict__['webframework']), \
        a.webframework, \
        type(a).__dict__['webframework'], \
        a.__dict__['webframework']
    print id(PythonSite2.__dict__['webframework']), \
        PythonSite2.webframework, \
        PythonSite2.__dict__['webframework']
except AttributeError as e:
    print e

Tornado
Tornado
Tornado
This is a read-only data descriptor.
Flask
59226032 Django Flask Django
59226032 Flask Flask


### Built-in function [property()](https://docs.python.org/2/library/functions.html#property)

`class property([fget[, fset[, fdel[, doc]]]])`

- Return a property attribute for new-style classes (classes that derive from object).


- fget is a function for getting an attribute value. 
- fset is a function for setting an attribute value. 
- fdel is a function for deleting an attribute value. 
- And doc creates a docstring for the attribute.


> - New in version 2.2.
- Changed in version 2.5: Use fget‘s docstring if no doc given.
- Changed in version 2.6: The getter, setter, and deleter attributes were added.

In [36]:
"""

Calling built-in function property() is a succinct way to build a data descriptor 
(property attribute) for a new style class:

    property(fget=None, fset=None, fdel=None, doc=None)
        -> property attribute
    
"""
class C(object):
    def getx(self): return self.__x
    def setx(self, value): self.__x = value
    def delx(self): del self.__x
    x = property(getx, setx, delx, "I'm the 'x' property.")
    def __init__(self):
        self.__x = 0
    
c = C()
print type(C.x)   # <type 'property'>
print C.x.__doc__ # I'm the 'x' property.
print c.x
c.x = 10
print c.x
del c.x
try:
    print c.x
except Exception as e:
    print type(e), e

<type 'property'>
I'm the 'x' property.
0
10
<type 'exceptions.AttributeError'> 'C' object has no attribute '_C__x'


#### @property

- If given, **doc** will be the docstring of the property attribute. 
- Otherwise, the property will copy fget‘s docstring (if it exists). 
- This makes it possible to create **read-only properties** easily using property() as a decorator:

In [32]:
class Parrot(object):
    def __init__(self):
        self._voltage = 100000

    @property
    def voltage(self):
        """Get the current voltage."""
        return self._voltage
    
p = Parrot()
print Parrot.voltage.__doc__
print p.voltage
try:
    p.voltage = 200000
except Exception as e:
    print type(e), e
try:
    del p.voltage
except Exception as e:
    print type(e), e

Get the current voltage.
100000
<type 'exceptions.AttributeError'> can't set attribute
<type 'exceptions.AttributeError'> can't delete attribute


#### getter, setter, and deleter Decorators

Turns the decorated functions into property attributes.


In [37]:
"""
using decorators
"""
class C(object):
    def __init__(self):
        self._x = None

    @property
    def x(self):
        """I'm the 'x' property."""
        return self._x

    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        del self._x
        
c = C()
print type(C.x)   # <type 'property'>
print C.x.__doc__ # I'm the 'x' property.
print c.x
c.x = 10
print c.x
del c.x
try:
    print c.x
except Exception as e:
    print type(e), e

<type 'property'>
I'm the 'x' property.
None
10
<type 'exceptions.AttributeError'> 'C' object has no attribute '_x'


In [38]:
"""
To see HOW property() is implemented in terms of the descriptor protocol, 
here is a pure Python equivalent
"""

class Property(object):
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
    
class C(object):
    def getx(self): return self.__x
    def setx(self, value): self.__x = value
    def delx(self): del self.__x
    x = Property(getx, setx, delx, "I'm the 'x' property.")
    def __init__(self):
        self.__x = 0
    
c = C()
print type(C.x)   # <class '__main__.Property'>, NOT <type 'property'>
print C.x.__doc__ # I'm the 'x' property.
print c.x
c.x = 10
print c.x
del c.x
try:
    print c.x
except Exception as e:
    print type(e), e

<class '__main__.Property'>
I'm the 'x' property.
0
10
<type 'exceptions.AttributeError'> 'C' object has no attribute '_C__x'


In [42]:
class Cell(object):
    """Simulate an spreadsheet cell"""
    def getvalue(self):
        print "Recalculate cell before returning value"
        self.recalc()
        return self._value
    
    value = property(getvalue)
    
    def __init__(self,colRowString='a1'):
        self._colRowString = colRowString
        self._value = 0
        
    def recalc(self):
        self._value += 1
        
c = Cell('b2')
print c.value
print c.value
print c.value
        
        

Recalculate cell before returning value
1
Recalculate cell before returning value
2
Recalculate cell before returning value
3


### [Functions and Methods](https://docs.python.org/2/howto/descriptor.html#functions-and-methods)

Python’s object oriented features are built upon a function based environment. 
Using non-data descriptors, the two are merged seamlessly.

- Class dictionaries store methods as functions. 
    
    - In a class definition, methods are written using def and lambda, the usual tools for creating functions. 
    - The only difference from regular functions is that the first argument is reserved for the object instance. 
    - By Python convention, the instance reference is called **_self_** but may be called this or any other variable name.

- To support method calls, 

    - functions include the **\_\_get\_\_()** method for binding methods during attribute access. 
    - This means that all functions are **non-data descriptors** which return bound or unbound methods depending whether they are invoked from an object or a class. 
    - In pure python, it works like this:
    
    ```
    class Function(object):
    . . .
    def __get__(self, obj, objtype=None):
        """Simulate func_descr_get() in Objects/funcobject.c"""
        return types.MethodType(self, obj, objtype)
    ```

In [100]:
class D(object):
     def f(self, x):
          return x
     def g(this):
          return 0
     def h(me,x):
          return x
     def j(x):
          return x

d = D()
print D.__dict__['f'] # Stored internally as a function
print D.f             # Get from a class becomes an unbound method
print D.f(d,'f1')     # f1
print d.f             # Get from an instance becomes a bound method
print d.g()           # this == self
print d.h(1)          # me   == self
print D.h(d,2)        # me   == d
try:
    print D.h(None,2) # <type 'exceptions.TypeError'>
except Exception as e:
    print type(e), e
try:
    print D.h(2)      # <type 'exceptions.TypeError'>
except Exception as e:
    print type(e), e
try:
    print D.j(1)      # <type 'exceptions.TypeError'>
except Exception as e:
    print type(e), e
print D.j(d)          # x == self ==> <__main__.D object at>


<function f at 0x0000000003932828>
<unbound method D.f>
f1
<bound method D.f of <__main__.D object at 0x00000000039E1C50>>
0
1
2
<type 'exceptions.TypeError'> unbound method h() must be called with D instance as first argument (got NoneType instance instead)
<type 'exceptions.TypeError'> unbound method h() must be called with D instance as first argument (got int instance instead)
<type 'exceptions.TypeError'> unbound method j() must be called with D instance as first argument (got int instance instead)


### Invoking Descriptors

- Direct Call

    `x.__get__(a)`
    
    
- Instance Binding

    a.x ==> `type(a).__dict__['x'].__get__(a, type(a))`


- Class Binding

    A.x ==> `A.__dict__['x'].__get__(None, A)`


- Super Binding

    - `super(B, obj).m()` searches `obj.__class__.__mro__` for the base class A immediately preceding B and then invokes the descriptor with the call: 
    - `A.__dict__['m'].__get__(obj, obj.__class__)`


### [Functions and Methods](https://docs.python.org/2/howto/descriptor.html#functions-and-methods)

continue ...

- The output suggests that bound and unbound methods are two different types. 
- But the actual C implementation of **PyMethod_Type** in ** *Objects/classobject.c* ** is a single object with two different representations depending on whether the **im_self** field is set or is NULL (the C equivalent of None):

    - If set (meaning **bound**), the original function (stored in the **im_func** field) is called as expected with the first argument set to the instance. 
    - If NULL, **all of the arguments** are passed **unchanged** to the original function. 
    
- The actual C implementation of **instancemethod_call()** is slightly more complex with some type checking.

### [Static Methods and Class Methods](https://docs.python.org/2/howto/descriptor.html#static-methods-and-class-methods)

<p>This chart summarizes the binding and its two most useful variants:</p>

<table border="1">
    <colgroup>
    <col width="30%" />
    <col width="39%" />
    <col width="32%" />
    </colgroup>
    <thead valign="bottom">
    <tr class="row-odd">
        <th class="head">Transformation</th>
        <th class="head">Called from an Object</th>
        <th class="head">Called from a Class</th>
    </tr>
    </thead>
    <tbody valign="top">
        <tr>
            <td>function</td>
            <td>f(obj, *args)</td>
            <td>f(*args)</td>
        </tr>
        <tr>
            <td>staticmethod</td>
            <td>f(*args)</td>
            <td>f(*args)</td>
        </tr>
        <tr>
            <td>classmethod</td>
            <td>f(type(obj), *args)</td>
            <td>f(klass, *args)</td>
        </tr>
    </tbody>
</table>

###### Static methods

- return the underlying function without changes. 
- Calling either c.f or C.f is the equivalent of 
    
    - `object.__getattribute__(c, "f")` or
    - `object.__getattribute__(C, "f")`
    
    
    - As a result, the function becomes identically accessible from either an object or a class.
    - Good candidates for static methods are methods that do not reference the self variable.


In [63]:
class E(object):
    
    def f(x):
          return x
    f = staticmethod(f)

print E.f(3)    # 3
print E().f(3)  # 3

class E(object):
    
    @staticmethod
    def f(x):
          return x

print E.f(3)    # 3
print E().f(3)  # 3

class StaticMethod(object):
    """Emulate PyStaticMethod_Type() in Objects/funcobject.c"""
    
    def __init__(self, f):
        self.f = f
    
    def __get__(self, obj, objtype=None):
        return self.f

class E(object):
    
    def f(x):
        return x
    f = StaticMethod(f)

print E.f(3)    # 3
print E().f(3)  # 3


3
3
3
3
3
3


##### Class Methods 

- prepend the class reference to the argument list before calling the function. 
- This format is the same for whether the caller is an object or a class:

In [74]:
"""
call built-in function classmethod()
"""
class E(object):
     def f(klass, x):
        return klass.__name__, x
     f = classmethod(f)

print E.f(3)   # ('E', 3)
print E().f(3) # ('E', 3)

"""
useful when a function only needs to have a class reference and does not care about any underlying data. 
e.g. to create alternate class constructors
"""
class Dict(object):
    def fromkeys(klass, iterable, value=None):
        """Emulate dict_fromkeys() in Objects/dictobject.c"""
        d = klass()
        for key in iterable:
            d[key] = value
        return d
    fromkeys = classmethod(fromkeys)
    
    def __init__(self):
        self._dict = dict()
        
    def __getitem__(self, key):
        if key in self._dict:
            return self._dict[key]
        return None
    
    def __setitem__(self, key, value):
        self._dict[key] = value
        
    def __str__(self):
        return str(self._dict)
    
print Dict.fromkeys(['a','b','c'], 1)
print Dict().fromkeys('abracadabra')

"""
use decorator
"""
class E(object):
    
    @classmethod
    def f(klass, x):
        return klass.__name__, x

print E.f(3)   # ('E', 3)
print E().f(3) # ('E', 3)

"""
simulation
"""
class ClassMethod(object):
    """Emulate PyClassMethod_Type() in Objects/funcobject.c"""
    def __init__(self, f):
        self.f = f
    
    def __get__(self, obj, klass=None): # class is type
        if klass is None:
            klass = type(obj)
            
        def newfunc(*args):
            return self.f(klass, *args)
          
        return newfunc
    
class E(object):
     def f(klass, x):
        return klass.__name__, x
     f = ClassMethod(f)

print E.f(3)   # ('E', 3)
print E().f(3) # ('E', 3)


('E', 3)
('E', 3)
{'a': 1, 'c': 1, 'b': 1}
{'a': None, 'r': None, 'b': None, 'c': None, 'd': None}
('E', 3)
('E', 3)
('E', 3)
('E', 3)


In [97]:
class _Missing(object):
    def __repr__(self):
        return 'no value'

    def __reduce__(self):
        return '_missing'


_missing = _Missing()

class cached_property(object):
    def __init__(self, func, name=None, doc=None):
        self.__name__ = name or func.__name__
        self.__module__ = func.__module__
        self.__doc__ = doc or func.__doc__
        self.func = func

    def __get__(self, obj, type=None):
        if obj is None:
            return self
        value = obj.__dict__.get(self.__name__, _missing)
        if value is _missing:
            value = self.func(obj)
            obj.__dict__[self.__name__] = value
        return value


class Foo(object):
    @cached_property
    def foo(self):
        print 'first calculate'
        result = 'this is result'
        return result


f = Foo()

print f.foo   # first calculate this is result
print f.foo   # this is result

first calculate
this is result
this is result
